# Tensium API Walker

Tensium API Walker allows Automation Engineers to build complex yet easily maintanable automated API tests.

This library makes it easy to quickly write automated API tests which require little to no maintenance (when using forward-feeding).

## How it works

Given a list of endpoints, and their possible inputs, Tensium will figure out how to exeute an API endpoint.

When chaining API calls, Tensium will forward feed the possible values to the next endpoint in the chain.

#### I.E

> We have two endpoints.

1. Get Weather Forecasts (Monday-Friday)
2. Update Weather Forecast (For any one given day)

> Tensium will step through both endpoints.
>
> Step 2 requires a body object containing forecast information.
>
> At step 1, possible values are forwarded to step 2 for endpoint execution.

# Dependencies

In [ ]:
import Tensium

from Tensium.api_management.TensiumEndpoint import TensiumEndpoint
from Tensium.api_management.TensiumAPIWalker import TensiumAPIWalker

# Get Weather Forecasts

First, let's take a look at a simple HTTP GET request in Tensium.

In [ ]:
forecasts = TensiumEndpoint(url='http://localhost:5120/weatherforecast',
                            possible_inputs=None, method='GET', regression=None)

# Update Weather Forecast

Now, try something a little more complex in nature.

Below, we will setup forward-fed HTTP POST request which will take inputs from the GET request we made earlier.

Let's start with setting up possible inputs.

# Possible Inputs

When specifying the possible inputs to an endpoint, you can either explicitly define these inputs, or you can have them forward-feed to your endpoint.


### Forward Feeding
Forward-feeding can be achieved by setting a wildcard ($) in the allowed_values option.

### Explicitly Defining
Simple specify a list of possible inputs

In [ ]:
possible_inputs = {
    'temperatureC': {
        'allowed_values': '$'
    },
    'summary': {
        'allowed_values': ['Hot Summer Day', 'Cold Summer Day']
    },
    'id': {
        'allowed_values': '$'
    },
    'date': {
        'allowed_values': '$'
    }
}

# Regression

Next, let's setup some simple regression testing.

With regression, we can validate that our endpoint had some effect on the previous endpoint.

For instance, when updating a forecast, our fetch endpoint should return a matching object to what was updated. (IE: new temperature, updated summary)

In [ ]:
# Do regression testing on fields 'temperatureC', 'date'

update_forecast_regression = {
    'inputs': ['temperatureC', 'date'],
    'input_key': 'id'
}

# Create Update Forecast Input

Let's create the update endpoint.

In [ ]:
update_forecast = TensiumEndpoint(url='http://localhost:5120/weatherforecast',
                                  possible_inputs=possible_inputs, method='POST', regression=update_forecast_regression)


# Walk the chain

Here is where the core functionality takes place.

Walk the chain, execute endpoints.

In [ ]:
tensium_walker = TensiumAPIWalker().walk_chain([forecasts, update_forecast])

In [ ]:
tensium_walker



1. Fetch HTTP [GET] http://localhost:5120/weatherforecast
    1. Return list({WeatherForecast})
    2. Forward feed random object list({WeatherForecast}) -> Update Forecast Endpoint
2. Update Forecast HTTP [POST] http://localhost:5120/weatherforecast
    1. As body, send forward-fed {WeatherForecast}
